In [2]:
import torch
import numpy as np
from torch.utils.data import WeightedRandomSampler, TensorDataset, DataLoader

numDataPoints = 1000
data_dim = 5
bs = 100

positive_class_proportion = 0.1
negative_class_proportion = 0.9

# Create dummy data with class imbalance 9 to 1
data = torch.FloatTensor(numDataPoints, data_dim)
target = np.hstack((np.zeros(int(numDataPoints * negative_class_proportion), dtype=np.int32),
                    np.ones(int(numDataPoints * positive_class_proportion), dtype=np.int32)))

print(f'target train 0/1: {len(np.where(target == 0)[0])}/{len(np.where(target == 1)[0])}')

class_sample_count = np.unique(target, return_counts=True)[1]
# weight = 1. / class_sample_count
# samples_weight = weight[target]

new_majority_proportion = 3
class_sample_count[0] /= new_majority_proportion
weight = 1. / class_sample_count
samples_weight = weight[target]

samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

target = torch.from_numpy(target).long()
train_dataset = TensorDataset(data, target)

train_loader = DataLoader(
    train_dataset, batch_size=bs, num_workers=1, sampler=sampler)

for i, (data, target) in enumerate(train_loader):
    print(f"batch index {i}, 0/1: {len(np.where(target.numpy() == 0)[0])}/{len(np.where(target.numpy() == 1)[0])}")

target train 0/1: 900/100
batch index 0, 0/1: 68/32
batch index 1, 0/1: 75/25
batch index 2, 0/1: 75/25
batch index 3, 0/1: 70/30
batch index 4, 0/1: 71/29
batch index 5, 0/1: 81/19
batch index 6, 0/1: 84/16
batch index 7, 0/1: 72/28
batch index 8, 0/1: 86/14
batch index 9, 0/1: 66/34


In [6]:
weights = 1. / torch.tensor(class_sample_count, dtype=torch.float)
samples_weights = weights[target]

sampler = WeightedRandomSampler(
    weights=samples_weights,
    num_samples=len(samples_weights),
    replacement=True)